# Clamped patterns
This notebook will be used to study how clamping works with the BCPNN. In short here I analyze:
* Whether the pattern converges to the clamped pattern for different parameters of the BCPNN (adapatation, bias, etc)

First we load all the usual libraries

In [1]:
from __future__ import print_function
import subprocess
import sys 
sys.path.append('../')

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec\

from mpl_toolkits.axes_grid1 import make_axes_locatable

from connectivity_functions import get_beta, get_w
from connectivity_functions import calculate_probability, calculate_coactivations
from data_transformer import build_ortogonal_patterns
from network import BCPNN

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

%matplotlib inline
sns.set(font_scale=2.5)
np.set_printoptions(suppress=True)

#### Git loading
Here we build the machinery so this notebook can be read in the future. If the notebook do not work with current version, just set the bolean value to truth and this will retrieve the last status from version control where the model was running.

In [2]:
# Git stuff

## Clamping for simple model
First we do a simple run just to see that we indeed get as the final state the model the clamping value. Or that the probabilities / activations of the model converge to the clamping values.

In [9]:
hypercolumns = 3
minicolumns = 3

patterns_dic = build_ortogonal_patterns(hypercolumns, minicolumns)
patterns = list(patterns_dic.values())

P = calculate_coactivations(patterns)
p = calculate_probability(patterns)

w = get_w(P, p)
beta = get_beta(p)

# Parameters and network intitiation
tau_z_post = 0.240
tau_z_pre = 0.240
g_a = 0
g_w = 1.0
g_beta = 1.0
tau_a = 2.7

dt = 0.01
T_simulation = 1.0
simulation_time = np.arange(0, T_simulation + dt, dt)

prng = np.random.RandomState(seed=0)

# Define the network
nn = BCPNN(hypercolumns, minicolumns, beta, w, p_pre=p, p_post=p, p_co=P,
           tau_z_post=tau_z_post, tau_z_pre=tau_z_pre,
           tau_a=tau_a, g_a=g_a, g_w=g_w, g_beta=g_beta, prng=prng, k=0)

# This is the training
I = np.array((1, 0, 0, 0, 1, 0, 0, 0, 1))
nn.randomize_pattern()
print('Initial pattern', nn.o)
nn.run_network_simulation(simulation_time, I=I)
print('Final pattern', nn.o)
print('I')
print('Difference', I - nn.o)
print('Error', np.linalg.norm(I - nn.o))

Initial pattern [ 0.5488135   0.71518937  0.60276338  0.54488318  0.4236548   0.64589411
  0.43758721  0.891773    0.96366276]
Final pattern [ 0.  0.  1.  0.  0.  1.  0.  0.  1.]
I
Difference [ 1. -0. -1. -0.  1. -1. -0. -0.  0.]
Error 2.0


In [6]:
print(I)
print(nn.o)
print(I - nn.o)

[1 0 0 0 1 0 0 0 1]
[ 0.  0.  1.  0.  0.  1.  0.  0.  1.]
[ 1. -0. -1. -0.  1. -1. -0. -0.  0.]


We see that the final pattern converges to the clamp value from the initial random pattern

## Systematic Testing for Biases
The following routine should test systematically the effect of the parameters for different values of the biases.

In [10]:
tolerance = 1e-5
I = np.array((1, 0, 0, 0, 1, 0, 0, 0, 1))

g_a_set = np.arange(0, 110, 10)
g_w_set = np.arange(0, 22, 2)
g_beta_set = np.arange(0, 22, 2)

error = np.zeros((g_a_set.size, g_beta_set.size, g_w_set.size))

for index_1, g_a in enumerate(g_a_set):
    for index_2, g_beta in enumerate(g_beta_set):
        for index_3, g_w in enumerate(g_w_set):

            nn = BCPNN(hypercolumns, minicolumns, beta, w, p_pre=p, p_post=p, p_co=P,
               g_w= g_w, g_beta=g_beta, g_a=g_a, prng=prng, k=0)

            nn.randomize_pattern()

            # This is the training

            nn.run_network_simulation(simulation_time, I=I)
            final = nn.o
            error[index_1, index_2, index_3] = np.linalg.norm(I - final)



print('Total failure', np.sum(error))

Total failure 2455.61611106


We see that the eror is very small a wide combination of bias, so the clamping mechanism is safe against this change of variables.

On the other hand if we decide to increase the number of hypercolumns and minicolumns 